In [ ]:
import os
import numpy as np
import rasterio
from rasterio.transform import from_origin
from rasterio import warp
import pandas as pd
from osgeo import gdal



def crop_geotiff(input_tiff_path, output_dir, crop_width, crop_height, output_geotiff):
    # Open the large GeoTIFF image
    ds = gdal.Open(input_tiff_path)
    gt = ds.GetGeoTransform()
    projection = ds.GetProjection()
    band = ds.GetRasterBand(1)
    No_data = band.GetNoDataValue()
    arr = np.array(band.ReadAsArray())
    print('Array', np.max(arr), np.min(arr), No_data)
    
    ds_LC = gdal.Open(output_geotiff)
    band_LC = ds_LC.GetRasterBand(1)
    arr_LC = np.array(band_LC.ReadAsArray())
    
    arr=np.where(arr_LC == 11, 0.0, arr) #arr[arr_LC==11] = 0.0
    # Create a new raster file to save the modified band
    output_tiff_path = os.path.join(output_dir,'target_no_waterbody.tif')
    driver = gdal.GetDriverByName("GTiff")
    out_ds = driver.Create(
        output_tiff_path,
        ds.RasterXSize,
        ds.RasterYSize,
        1,  # Number of bands
        gdal.GDT_Float32,  # Data type
    )

    # Set the GeoTransform and Projection for the output file
    out_ds.SetGeoTransform(gt)
    out_ds.SetProjection(projection)

    # Write the modified data to the new file
    out_band = out_ds.GetRasterBand(1)
    out_band.WriteArray(arr)
    # print(np.sum(arr))
    out_band.SetNoDataValue(No_data)
    
    ds = gdal.Open(output_tiff_path)
    gt = ds.GetGeoTransform()
    projection = ds.GetProjection()
    band = ds.GetRasterBand(1)
    
    arr=np.where(arr == No_data, np.nan, arr) #arr[arr==No_data ] = np.nan
    print('Array', np.max(arr), np.min(arr), No_data)
    img_width = ds.RasterXSize
    img_height = ds.RasterYSize

    # Calculate number of tiles in x and y directions
    x_tiles = img_width // crop_width
    y_tiles = img_height // crop_height
    count = 1
    for i in range(x_tiles):
        for j in range(y_tiles):
            # Calculate pixel coordinates for the tile
            offset_x = i * crop_width
            offset_y = j * crop_height
            
            # Read the data for the tile
            tile_data = band.ReadAsArray(offset_x, offset_y, crop_width, crop_height)
            tile_data=np.where(tile_data == No_data, np.nan, tile_data)
            
            
            # if np.sum(np.isnan(tile_data))>0.0 or np.sum(tile_data)==0.0: #(np.sum(tile_data)/(crop_width*crop_height))<0.00001
            #     # print(np.sum(tile_data)/(crop_width*crop_height))
            #     # print(np.max(tile_data))
            #     continue
            # else:
            if np.sum(tile_data)>0.0:
                print(np.max(tile_data), np.min(tile_data))

            
                # Create the cropped output file
                driver = gdal.GetDriverByName('GTiff')
                output_file = os.path.join(output_dir, raster_path.split("/")[-1][:-4]+f'_crop_{count}.tif')
                out_ds = driver.Create(output_file, crop_width, crop_height, 1, band.DataType)
                
                # Set GeoTransform and Projection for the cropped file
                new_gt = (
                    gt[0] + offset_x * gt[1], gt[1], gt[2],
                    gt[3] + offset_y * gt[5], gt[4], gt[5]
                )
                out_ds.SetGeoTransform(new_gt)
                out_ds.SetProjection(ds.GetProjection())
                
                
                # Write the cropped data to the output file
                out_ds.GetRasterBand(1).WriteArray(tile_data)
                # out_ds.SetNoDataValue(No_data)
                out_ds.FlushCache()
                
                # Clean up
                out_ds = None
                count += 1

crop_width = 512                       # Width of each cropped image (in pixels)
crop_height = 512  

# Input files and directories
events_file = '/usr/workspace/lazin1/anaconda_dane/envs/RAPID/EVENTS/combined.csv'
combined_df = pd.read_csv(events_file, header=None)

for idx, raster_path in enumerate(combined_df[0]): 
    event = raster_path.split("/")[-1][:-4]  # Extract event name from raster path
    TARGET_RASTER_DIR = f"/p/lustre1/lazin1/RAPID_Archive_Flood_Maps/Tile_No_waterbody/{event}"
    
    if os.path.exists(TARGET_RASTER_DIR):
        print('Exists', TARGET_RASTER_DIR)
        # continue
    else:
        print('New', TARGET_RASTER_DIR)
        os.makedirs(TARGET_RASTER_DIR, exist_ok=True)

    # Output directory for cropped raster
    cropped_output_dir = TARGET_RASTER_DIR
    temp_raster_path = f"/p/lustre2/lazin1/Annual_NLCD_LndCov_2023_CU_C1V0.tif"
    output_geotiff = os.path.join(cropped_output_dir, 'LC_' + event + '.tif')
    reference_raster = raster_path

    # Process the reference raster
    with rasterio.open(reference_raster) as ref:
        ref_transform = ref.transform
        ref_crs = ref.crs
        ref_shape = (ref.height, ref.width)
        ref_bounds = ref.bounds
        ref_nodata = ref.nodata  # Get NoData value from reference raster

    # Reproject and resample to match the reference raster
    with rasterio.open(temp_raster_path) as src:
        with rasterio.open(
            output_geotiff,
            "w",
            driver="GTiff",
            height=ref_shape[0],
            width=ref_shape[1],
            count=1,
            dtype="float32",
            crs=ref_crs,
            transform=ref_transform,
            nodata=ref_nodata  # Use the same NoData value as the reference raster
        ) as dst:
            # Create an array for the destination
            destination_array = np.empty((ref_shape[0], ref_shape[1]), dtype="float32")
            destination_array.fill(ref_nodata)  # Initialize with NoData value
            

            # Reproject the data
            warp.reproject(
                source=rasterio.band(src, 1),
                destination=destination_array,
                src_transform=src.transform,
                src_crs=src.crs,
                dst_transform=ref_transform,
                dst_crs=ref_crs,
                resampling=warp.Resampling.nearest
            )

            # Mask with NoData where reference has NoData
            with rasterio.open(reference_raster) as ref:
                reference_data = ref.read(1)  # Read reference raster band
                destination_array[reference_data == ref_nodata] = ref_nodata  # Apply NoData mask
                # destination_array[destination_array == 11] = ref_nodata

            # Write to the output raster
            dst.write(destination_array, 1)


    print(f"Processed and saved: {output_geotiff}")
    try:
        print(f"Processing {raster_path}...")
        crop_geotiff(raster_path, TARGET_RASTER_DIR, crop_width, crop_height, output_geotiff)
    except:
        continue
    
    os.remove(output_geotiff)


In [11]:

import os
import numpy as np
import rasterio
from rasterio.transform import from_origin
from rasterio import warp
import pandas as pd
from osgeo import gdal
file = '/p/lustre1/lazin1/RAPID_Archive_Flood_Maps/Tile_No_waterbody/flood_WM_S1A_IW_GRDH_1SDV_20180919T231415_20180919T231440_023774_0297CC_374E/target_no_waterbody.tif'

ds = gdal.Open(file)
gt = ds.GetGeoTransform()
projection = ds.GetProjection()
band = ds.GetRasterBand(1)
array = band.ReadAsArray()
print(np.nanmax(array))

255


In [ ]:
import rasterio
import numpy as np

# Define the GeoTIFF file path
geotiff_path = f"/p/lustre1/lazin1/RAPID_Archive_Flood_Maps/Tile_No_waterbody/flood_WM_S1A_IW_GRDH_1SDV_20180919T231415_20180919T231440_023774_0297CC_374E/target_no_waterbody.tif"

# Open the GeoTIFF
with rasterio.open(geotiff_path) as src:
    # Read the first band as a NumPy array (assuming single-band raster)
    raster_array = src.read(1)

    # Get raster dimensions
    rows, cols = raster_array.shape

    # Define the patch size
    patch_size = 512

    # Initialize variables to track the maximum sum and patch coordinates
    max_sum = -np.inf
    max_coords = (0, 0)  # Top-left corner of the maximum-sum patch

    # Iterate over the raster using a sliding window of size 512x512
    for i in range(0, rows - patch_size + 1):
        for j in range(0, cols - patch_size + 1):
            # Extract the patch
            patch = raster_array[i:i + patch_size, j:j + patch_size]

            # Calculate the sum of the patch
            patch_sum = np.sum(patch)

            # Update the maximum sum and coordinates if a new maximum is found
            if patch_sum > max_sum:
                max_sum = patch_sum
                max_coords = (i, j)

    # Print the results
    print(f"Maximum sum: {max_sum}")
    print(f"Top-left corner of the maximum-sum patch: Row={max_coords[0]}, Col={max_coords[1]}")


In [ ]:
a = np.array([0 , 1 , np.nan])
np.sum(a)

In [ ]:
import os
from osgeo import gdal
import numpy as np
import pandas as pd

def crop_geotiff(input_tiff_path, output_dir, crop_width, crop_height):
    # Open the large GeoTIFF image
    ds = gdal.Open(input_tiff_path)
    gt = ds.GetGeoTransform()
    band = ds.GetRasterBand(1)
    No_data = band.GetNoDataValue()
    arr = np.array(band.ReadAsArray())
    # print('Array', np.max(arr), np.min(arr), No_data)
    img_width = ds.RasterXSize
    img_height = ds.RasterYSize

    # Calculate number of tiles in x and y directions
    x_tiles = img_width // crop_width
    y_tiles = img_height // crop_height
    count = 1
    for i in range(x_tiles):
        for j in range(y_tiles):
            # Calculate pixel coordinates for the tile
            offset_x = i * crop_width
            offset_y = j * crop_height
            
            # Read the data for the tile
            tile_data = band.ReadAsArray(offset_x, offset_y, crop_width, crop_height)
            tile_data=np.where(tile_data == No_data, np.nan, tile_data)
            
            if np.sum(np.isnan(tile_data))>0:
                continue
            else:
                # print(np.max(tile_data), np.min(tile_data))

            
                # Create the cropped output file
                driver = gdal.GetDriverByName('GTiff')
                output_file = os.path.join(output_dir, raster_path.split("/")[-1][:-4]+f'_crop_{count}.tif')
                out_ds = driver.Create(output_file, crop_width, crop_height, 1, band.DataType)
                
                # Set GeoTransform and Projection for the cropped file
                new_gt = (
                    gt[0] + offset_x * gt[1], gt[1], gt[2],
                    gt[3] + offset_y * gt[5], gt[4], gt[5]
                )
                out_ds.SetGeoTransform(new_gt)
                out_ds.SetProjection(ds.GetProjection())
                
                
                # Write the cropped data to the output file
                out_ds.GetRasterBand(1).WriteArray(tile_data)
                # out_ds.SetNoDataValue(No_data)
                out_ds.FlushCache()
                
                # Clean up
                out_ds = None
                count += 1

# Parameters
# input_tiff_path = '/p/lustre1/lazin1/RAPID_Archive_Flood_Maps/20170829/flooding_S1A_IW_GRDH_1SDV_20170829T002620_20170829T002645_018131_01E74D_D734/flood_WM_S1A_IW_GRDH_1SDV_20170829T002620_20170829T002645_018131_01E74D_D734.tif'  # Path to the large GeoTIFF image
           # Folder to save cropped images
crop_width = 512                       # Width of each cropped image (in pixels)
crop_height = 512                     # Height of each cropped image (in pixels)

# Create output directory if it doesn't exist
# EVENT_STR = "Harvey_20170829_D734_non_flood"
# EVENT_STRS = ["Harvey_20170829_D734_non_flood","Harvey_20170831_9366_non_flood" , "Harvey_20170831_0776_non_flood", "Harvey_20170829_B8C4_non_flood", "Harvey_20170829_3220_non_flood"]

# EVENT_STRS = ["Harvey_20170829_D734_non_flood","Harvey_20170831_9366_non_flood" , "Harvey_20170831_0776_non_flood", "Harvey_20170829_B8C4_non_flood", "Harvey_20170829_3220_non_flood"]
# EVENT_STRS = ["Florence_20180919_374E_non_flood", "Florence_20180919_B86C_non_flood"]

####### WHEN EVENT_Mississippi_3AC6_non_flood.csv type file is privided

# EVENT_STRS= ["All"]



# # Harvey_20170829_3220_non_flood.csv #Harvey_20170829_B8C4_non_flood.csv  #Harvey_20170831_0776_non_flood.csv #Harvey_20170831_9366_non_flood.csv #Harvey_20170829_D734_non_flood.csv
# #"Mississippi_20190617_B9D7_non_flood" #"Mississippi_20190617_B9D7_non_flood"  # "Mississippi_20190617_42BF_non_flood"  #"Mississippi_20190617_3AC6_non_flood" #"Mississippi_20190617_9D85_non_flood" #"Mississippi_20190617_5E5F_non_flood"
# for EVENT_STR in EVENT_STRS:
#     RASTER_PATHS = pd.read_csv(f"/usr/workspace/lazin1/anaconda_dane/envs/RAPID/EVENT_{EVENT_STR}.csv", header=None) #'/usr/workspace/lazin1/anaconda_dane/envs/RAPID/flood_img_list_flood_non_flood_test2.csv')  #/usr/workspace/lazin1/anaconda_dane/envs/RAPID/EVENT_MISSISSIPPI_non_flood_20190617_3AC6.csv #/usr/workspace/lazin1/anaconda_dane/envs/RAPID/EVENT_MISSISSIPPI_non_flood_20190617_C310.csv  #/usr/workspace/lazin1/anaconda_dane/envs/RAPID/EVENT_MISSISSIPPI_non_flood_20190617_B9D7.csv
#     # print(RASTER_PATHS)






# # Harvey_20170829_3220_non_flood.csv #Harvey_20170829_B8C4_non_flood.csv  #Harvey_20170831_0776_non_flood.csv #Harvey_20170831_9366_non_flood.csv #Harvey_20170829_D734_non_flood.csv

# #"Mississippi_20190617_B9D7_non_flood" #"Mississippi_20190617_B9D7_non_flood"  # "Mississippi_20190617_42BF_non_flood"  #"Mississippi_20190617_3AC6_non_flood" #"Mississippi_20190617_9D85_non_flood" #"Mississippi_20190617_5E5F_non_flood"


# # RASTER_PATHS = pd.read_csv(f"/usr/workspace/lazin1/anaconda_dane/envs/RAPID/EVENT_{EVENT_STR}.csv"), header=None) #'/usr/workspace/lazin1/anaconda_dane/envs/RAPID/flood_img_list_flood_non_flood_test2.csv')  #/usr/workspace/lazin1/anaconda_dane/envs/RAPID/EVENT_MISSISSIPPI_non_flood_20190617_3AC6.csv #/usr/workspace/lazin1/anaconda_dane/envs/RAPID/EVENT_MISSISSIPPI_non_flood_20190617_C310.csv  #/usr/workspace/lazin1/anaconda_dane/envs/RAPID/EVENT_MISSISSIPPI_non_flood_20190617_B9D7.csv

#     raster_paths= RASTER_PATHS.to_numpy()

#     for idx, raster_path in enumerate(raster_paths):
#         output_dir = f"/p/lustre1/lazin1/RAPID_Archive_Flood_Maps/Tile_No_Threshold_{EVENT_STR}"
#         os.makedirs(output_dir, exist_ok=True)
#         try:
#             print(f"Processing {raster_path[0]}...")
#             crop_geotiff(raster_path[0], output_dir, crop_width, crop_height)
#         except:
#             continue
        
        
  ####### WHEN EVENT_Mississippi_3AC6_non_flood.csv type file is privided      
        
events_file = '/usr/workspace/lazin1/anaconda_dane/envs/RAPID/EVENTS/combined.csv'
combined_df = pd.read_csv(events_file, header=None) 
for idx, raster_path in enumerate(combined_df[0]): #events = [raster_path.split("/")[-1][:-4] 
    # print(raster_path)
    dir = raster_path.split("/")[-1][:-4]
    output_dir = f"/p/lustre1/lazin1/RAPID_Archive_Flood_Maps/Tile_{dir}"
    os.makedirs(output_dir, exist_ok=True)
    try:
        print(f"Processing {raster_path}...")
        crop_geotiff(raster_path, output_dir, crop_width, crop_height)
    except:
        continue
    
    
    


In [ ]:
import rasterio
import numpy as np

# Define the GeoTIFF file path
geotiff_path = "input.tif"

# Open the GeoTIFF
with rasterio.open(geotiff_path) as src:
    # Read the first band as a NumPy array (assuming single-band raster)
    raster_array = src.read(1)

    # Get raster dimensions
    rows, cols = raster_array.shape

    # Define the patch size
    patch_size = 512

    # Initialize variables to track the maximum sum and patch coordinates
    max_sum = -np.inf
    max_coords = (0, 0)  # Top-left corner of the maximum-sum patch

    # Iterate over the raster using a sliding window of size 512x512
    for i in range(0, rows - patch_size + 1):
        for j in range(0, cols - patch_size + 1):
            # Extract the patch
            patch = raster_array[i:i + patch_size, j:j + patch_size]

            # Calculate the sum of the patch
            patch_sum = np.sum(patch)

            # Update the maximum sum and coordinates if a new maximum is found
            if patch_sum > max_sum:
                max_sum = patch_sum
                max_coords = (i, j)

    # Print the results
    print(f"Maximum sum: {max_sum}")
    print(f"Top-left corner of the maximum-sum patch: Row={max_coords[0]}, Col={max_coords[1]}")
